# Dataset bruto

descrição do dataset escolhido, bem como sua sua origem

In [ ]:
import requests, warnings
import datatable as dt
from tqdm import tqdm

In [2]:
def download_raw_data(
    url_base,
    from_year,
    to_year,
    output_dir=''
  ):
  years = range(from_year, to_year + 1)
  n_years = to_year - from_year + 1
  for y in tqdm(years, total=n_years):
    fname = f'censo_{y}.zip'
    export = output_dir + fname
    url = url_base.format(year = y)
    try:
      with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        r = requests.get(
          url, verify = False)
      with open(export, 'wb') as f:
        f.write(r.content)
      print(f'Success to download year: {y}')
    except:
      print(f'Failed to download year: {y} from {url}')


## Fonte

Instituto Nacional de Estudos e Pesquisas Educacionais Anísio Teixeira (Inep)

[Censo da educação superior](https://www.gov.br/inep/pt-br/areas-de-atuacao/pesquisas-estatisticas-e-indicadores/censo-da-educacao-superior/resultados)

In [3]:
url_base = '/'.join([
  'https://download.inep.gov.br',
  'microdados',
  'microdados_censo_da_educacao_superior_{year}.zip'])
url_base

'https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_{year}.zip'

In [4]:
from_year = 2010
to_year = 2019

In [5]:
output_dir = '../data/raw/'

In [6]:
download_raw_data(
  url_base,
  from_year,
  to_year,
  output_dir
)

 10%|█         | 1/10 [00:04<00:37,  4.15s/it]

Success to download year: 2010


 20%|██        | 2/10 [00:13<00:57,  7.20s/it]

Success to download year: 2011


 30%|███       | 3/10 [00:20<00:49,  7.08s/it]

...

Success to download year: 2017


 90%|█████████ | 9/10 [01:33<00:11, 11.11s/it]

Success to download year: 2018


100%|██████████| 10/10 [01:48<00:00, 10.81s/it]

Success to download year: 2019
